In [3]:
import matplotlib.pyplot as plt
import os, glob
import numpy as np
from scipy.fft import fft, fftfreq
import scipy.signal
import pywt
from scipy.signal import butter, lfilter

In [4]:
def read_file(filename):
    
    with open(filename, "r") as file:
        piezo_set = []
        mic_set = []
        for line in file:
            stripped_line = line.strip()
            if stripped_line:  
                try:
                    # try to convert the line to an integer
                    piezo_data, mic_data = stripped_line.split(',') # data format: piezo, mic
                    piezo_set.append(int(piezo_data))
                    mic_set.append(int(mic_data))
                except ValueError:
                    # handle the case where the line contains non-integer values
                    parts = stripped_line.split('\x00')
                    for part in parts:
                        if part:  
                            try:
                                piezo_data, mic_data = stripped_line.split(',') # data format: piezo, mic
                                piezo_set.append(int(piezo_data))
                                mic_set.append(int(mic_data))
                            except ValueError:
                                print(f"warning: '{part}' cannot be converted to an integer")
    piezo_set = np.array(piezo_set)
    mic_set = np.array(mic_set)
    denoised_piezo_data = wavelet_denoise(piezo_set, wavelet='db1', level=1) # 这里使用了Daubechies小波（'db1'），也就是Haar小波，分解层数为1。
    denoised_mic_data = wavelet_denoise(mic_set, wavelet='db1', level=1)
    return denoised_piezo_data, denoised_mic_data
    # return piezo_set, mic_set

def signal_visualization(piezo, mic, filename):
    plt.figure()
    plt.plot(piezo, linestyle='-', label='piezo Data') #可视化一个拳头敲击周期的动作
    plt.plot(mic, linestyle='-', label='mic Data')
    plt.title(f'Sensor Data Over Time for {filename}')
    plt.xlabel('Time (arbitrary units)')
    plt.ylabel('Sensor Value')
    # plt.ylim(0,6000)
    plt.legend()
    plt.grid(True)
    plt.show()

def fft_visualization(signal, fft_result, filename):

    fs = 6000
    
    frequencies = np.fft.fftfreq(len(fft_result), 1/fs)
    magnitude = np.abs(fft_result)

    plt.figure(figsize=(14, 7))
    plt.plot(frequencies, magnitude)
    plt.title(f'FFT of {filename}')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.grid(True)
    plt.xlim(0, fs / 2)  # Limit x-axis to positive frequencies only up to Nyquist frequency
    plt.show()

    return frequencies, magnitude

def fft_features(fft_results):
    # Assume fft_results is an array of FFT coefficients
    magnitude_spectrum = np.abs(fft_results)
    
    # Spectral Centroid： Indicates the "center of mass" of the spectrum, giving an idea of the "brightness" of a sound.
    spectral_centroid = np.sum(magnitude_spectrum * np.arange(len(magnitude_spectrum))) / np.sum(magnitude_spectrum)
    
    # Spectral Rolloff：The frequency below which a certain percentage of the total spectral energy (commonly 85% or 95%) is contained, which helps in differentiating between harmonic content and noise.
    spectral_rolloff_threshold = 0.85 * np.sum(magnitude_spectrum)
    cumulative_sum = np.cumsum(magnitude_spectrum)
    spectral_rolloff = np.where(cumulative_sum >= spectral_rolloff_threshold)[0][0]
    
    # Spectral Flux：Measures the amount of local spectral change between successive frames, useful for detecting events.
    spectral_flux = np.sum((np.diff(magnitude_spectrum) ** 2))
    
    # Total Spectral Energy：Sum of squares of the FFT coefficients can serve as a measure of the overall signal energy.
    total_spectral_energy = np.sum(magnitude_spectrum ** 2)
    
    return {
        'spectral_centroid': spectral_centroid,
        'spectral_rolloff': spectral_rolloff,
        'spectral_flux': spectral_flux,
        'total_spectral_energy': total_spectral_energy
    }

def wavelet_features(signal):
    wavelet_transform = scipy.signal.cwt(signal, scipy.signal.ricker, widths=np.arange(1, 31))
    wavelet_energy = np.sum(wavelet_transform**2, axis=0)
    
    features = {
        "wavelet_energy": wavelet_energy
    }
    return features



def wavelet_denoise(data, wavelet, level):
    # 小波变换
    coeff = pywt.wavedec(data, wavelet, mode='per', level=level)
    
    # 计算阈值
    sigma = np.median(np.abs(coeff[-1])) / 0.6745
    threshold = sigma * np.sqrt(2 * np.log(len(data)))
    
    # 阈值处理
    coeff[1:] = [pywt.threshold(i, value=threshold, mode='soft') for i in coeff[1:]]
    
    # 重构信号
    reconstructed_signal = pywt.waverec(coeff, wavelet, mode='per')
    return reconstructed_signal

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = scipy.signal.butter(order, [low, high], btype='band')
    y = scipy.signal.lfilter(b, a, data)
    return y

def butter_bandstop_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = scipy.signal.butter(order, [low, high], btype='bandstop')
    y = scipy.signal.lfilter(b, a, data)
    return y

def load_data():
    features, labels = [], []
    # count = 0
    for file in glob.glob("data/combine/*/*.txt"):
        file_name = os.path.basename(file)
        label = os.path.dirname(file).split("/")[2]
        # print(file_name,label)
        piezo_signal, mic_signal = read_file(file)
        # piezo_signal_centered = piezo_signal - np.mean(piezo_signal)
        # mic_signal_centered = mic_signal - np.mean(mic_signal)
        
        piezo_fft_result = np.fft.fft(piezo_signal)
        mic_fft_result = np.fft.fft(mic_signal)
        # print(wavelet_features(piezo_signal))
        # print(fft_features(piezo_fft_result))

        piezo_spectral_feature = fft_features(piezo_fft_result)
        piezo_spectral_feature = np.array(list(piezo_spectral_feature.values()))
        piezo_wavelet_feature = wavelet_features(piezo_signal)['wavelet_energy']
        piezo_feature = np.concatenate((piezo_spectral_feature, piezo_wavelet_feature))
        # print(piezo_spectrial_feature_values)

        mic_spectral_feature = fft_features(mic_fft_result)
        mic_spectral_feature = np.array(list(mic_spectral_feature.values()))
        mic_wavelet_feature = wavelet_features(mic_signal)['wavelet_energy']
        mic_feature = np.concatenate((mic_spectral_feature, mic_wavelet_feature))

        combined_feature = np.concatenate((piezo_feature, mic_feature))

        features.append(combined_feature)
        labels.append(label)
    # print(features)
    return features, labels

In [85]:
# return features from a single file, method similar to load_data function
def load_single_data(file):
    piezo_signal, mic_signal = read_file(file)
    piezo_fft_result = np.fft.fft(piezo_signal)
    mic_fft_result = np.fft.fft(mic_signal)

    piezo_spectral_feature = fft_features(piezo_fft_result)
    piezo_spectral_feature = np.array(list(piezo_spectral_feature.values()))
    piezo_wavelet_feature = wavelet_features(piezo_signal)['wavelet_energy']
    piezo_feature = np.concatenate((piezo_spectral_feature, piezo_wavelet_feature))

    mic_spectral_feature = fft_features(mic_fft_result)
    mic_spectral_feature = np.array(list(mic_spectral_feature.values()))
    mic_wavelet_feature = wavelet_features(mic_signal)['wavelet_energy']
    mic_feature = np.concatenate((mic_spectral_feature, mic_wavelet_feature))

    features = np.concatenate((piezo_feature, mic_feature))

    return features

# call load_single_data function to get features from a single file, e.g. "data/combine/finger-snap/1714366681_1s_data.txt"
file = "data/combine/finger-snap/1714366682_1s_data.txt"
features = load_single_data(file)

/var/folders/0c/21w8m2ps1nj3wp8gv22kpyp40000gn/T/ipykernel_49319/870808706.py:88: DeprecationWarning: scipy.signal.cwt is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  wavelet_transform = scipy.signal.cwt(signal, scipy.signal.ricker, widths=np.arange(1, 31))


In [86]:
features[:10]

array([3.56701530e+02, 0.00000000e+00, 1.08927598e+14, 1.08891028e+14,
       1.21093504e+06, 1.73582241e+07, 4.44390901e+07, 7.83110036e+07,
       1.15246691e+08, 1.53512191e+08])

In [389]:
# features = load_data()[0]

# # Initialize a figure with subplots
# fig, axs = plt.subplots(4, 1, figsize=(10, 12))  # 4 rows, 1 column for each feature
# fig.suptitle('Comparison of Spectral Features Across Signals')

# # Names of features for plotting
# feature_names = ['spectral_centroid', 'spectral_rolloff', 'spectral_flux', 'total_spectral_energy']

# # Loop over each feature to create a subplot for that feature
# for i, feature in enumerate(feature_names):
#     # Extract the feature across all signals
#     values = [signal[feature] for signal in features]

#     # Plotting
#     axs[i].plot(values, marker='o', linestyle='-')  # Line plot
#     axs[i].set_title(feature)
#     axs[i].set_xlabel('Signal Index')
#     axs[i].set_ylabel('Value')

# # Adjust layout
# plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust the rect to fit the overall title
# plt.show()


In [390]:
# piezo_signal, mic_signal = read_file("data/1714360224_1s_data.txt")
# fs = 6000

# filtered_piezo = butter_bandstop_filter(piezo_signal, 60, 80, fs)
# filtered_piezo = butter_bandstop_filter(filtered_piezo, 200, 250, fs)

# filtered_mic = butter_bandstop_filter(mic_signal, 60, 80, fs)
# filtered_mic = butter_bandstop_filter(filtered_mic, 200, 250, fs)

# fft_piezo = np.fft.fft(piezo_signal)
# fft_mic = np.fft.fft(mic_signal)
# fft_filtered_piezo = np.fft.fft(filtered_piezo)
# fft_filtered_mic = np.fft.fft(filtered_mic)



# fft_freqs_piezo = np.fft.fftfreq(len(fft_piezo), 1/fs)
# fft_magnitude_piezo = np.abs(fft_piezo)
# fft_freqs_filtered_piezo = np.fft.fftfreq(len(fft_filtered_piezo), 1/fs)
# fft_magnitude_filtered_piezo = np.abs(fft_filtered_piezo)

# fft_freqs_mic = np.fft.fftfreq(len(fft_mic), 1/fs)
# fft_magnitude_mic = np.abs(fft_mic)
# fft_freqs_filtered_mic = np.fft.fftfreq(len(fft_filtered_mic), 1/fs)
# fft_magnitude_filtered_mic = np.abs(fft_filtered_mic)

# plt.figure(figsize=(14, 7))
# plt.subplot(2, 1, 1)
# plt.plot(fft_freqs_piezo, fft_magnitude_piezo)
# plt.plot(fft_freqs_filtered_piezo, fft_magnitude_filtered_piezo)
# plt.title('FFT of Piezo Signal')
# plt.xlabel('Frequency (Hz)')
# plt.ylabel('Magnitude')
# plt.grid(True)
# plt.xlim(0,fs/2)  # Limit x-axis to positive frequencies only up to Nyquist frequency
# plt.ylim(0, 10000)  # Limit y-axis to positive frequencies only up to Nyquist frequency

# plt.subplot(2, 1, 2)
# plt.plot(fft_freqs_mic, fft_magnitude_mic)
# plt.plot(fft_freqs_filtered_mic, fft_magnitude_filtered_mic)
# plt.title('FFT of Mic Signal')
# plt.xlabel('Frequency (Hz)')
# plt.ylabel('Magnitude')
# plt.grid(True)
# plt.xlim(0, fs/2)  # Limit x-axis to positive frequencies only up to Nyquist frequency
# plt.ylim(0, 10000)  # Limit y-axis to positive frequencies only up to Nyquist frequency

# plt.tight_layout()
# plt.show()

In [391]:
# import os, glob
# from sklearn.preprocessing import MinMaxScaler, StandardScaler

# finger_multiple_piezo, finger_multiple_mic = read_file("data/combine/finger-multiple/1714278475_1s_data.txt")
# finger_single_piezo, finger_single_mic = read_file("data/combine/finger-single/1714342361_1s_data.txt")
# no_gesture_piezo, no_gesture_mic = read_file("data/combine/no-gesture/1714347546_1s_data.txt")

# min_length = min(len(finger_multiple_piezo), len(finger_multiple_mic), len(finger_single_piezo), len(finger_single_mic), len(no_gesture_piezo), len(no_gesture_mic))
# finger_multiple_piezo = finger_multiple_piezo[:min_length]
# finger_multiple_mic = finger_multiple_mic[:min_length]
# finger_single_piezo = finger_single_piezo[:min_length]
# finger_single_mic = finger_single_mic[:min_length]
# no_gesture_piezo = no_gesture_piezo[:min_length]
# no_gesture_mic = no_gesture_mic[:min_length]

# fs = 6000
# fft_finger_multiple_piezo = np.fft.fft(finger_multiple_piezo)
# fft_finger_multiple_mic = np.fft.fft(finger_multiple_mic)
# fft_finger_single_piezo = np.fft.fft(finger_single_piezo)
# fft_finger_single_mic = np.fft.fft(finger_single_mic)
# fft_no_gesture_piezo = np.fft.fft(no_gesture_piezo)
# fft_no_gesture_mic = np.fft.fft(no_gesture_mic)

# freq_bins = np.fft.fftfreq(n=len(finger_multiple_piezo), d=1/fs)


# magnitude_finger_multiple_piezo = np.abs(fft_finger_multiple_piezo)
# magnitude_finger_multiple_mic = np.abs(fft_finger_multiple_mic)
# magnitude_finger_single_piezo = np.abs(fft_finger_single_piezo)
# magnitude_finger_single_mic = np.abs(fft_finger_single_mic)
# magnitude_no_gesture_piezo = np.abs(fft_no_gesture_piezo)
# magnitude_no_gesture_mic = np.abs(fft_no_gesture_mic)

# plt.figure(figsize=(12, 6))
# plt.subplot(2, 1, 1)
# plt.plot(freq_bins, magnitude_finger_multiple_piezo, label='Finger Multiple Piezo FFT')
# plt.plot(freq_bins, magnitude_finger_single_piezo, label='Finger Single Piezo FFT')
# plt.plot(freq_bins, magnitude_no_gesture_piezo, label='No Gesture Piezo FFT')
# plt.title('Piezo FFT Comparison')
# plt.xlabel('Frequency (Hz)')
# plt.ylabel('Magnitude')
# plt.xlim(0, fs/2)  # Limit x-axis to positive frequencies only up to Nyquist frequency
# plt.ylim(0, 100000)
# plt.legend()

# plt.subplot(2, 1, 2)
# plt.plot(freq_bins, magnitude_finger_multiple_mic, label='Finger Multiple Mic FFT')
# plt.plot(freq_bins, magnitude_finger_single_mic, label='Finger Single Mic FFT')
# plt.plot(freq_bins, magnitude_no_gesture_mic, label='No Gesture Mic FFT')
# plt.title('Mic FFT Comparison')
# plt.xlabel('Frequency (Hz)')
# plt.ylabel('Magnitude')
# plt.xlim(0, fs/2)  # Limit x-axis to positive frequencies only up to Nyquist frequency
# plt.ylim(0, 100000)  # Limit y-axis to better visualize the differences
# plt.legend()

# plt.tight_layout()
# plt.show()






In [9]:
def pad_features(features):
    max_length = max(len(f) for f in features)
    padded_features = np.array([np.pad(f, (0, max_length - len(f)), 'constant') for f in features])
    return padded_features

In [10]:
from sklearn.preprocessing import StandardScaler

# Assuming load_data() loads your features and labels
features, labels = load_data()
padded_features = pad_features(features)

# Now padded_features should be convertible to a numpy array
features_array = np.array(padded_features)
labels_array = np.array(labels)

# Proceed with scaling or other ML preparations
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_array)

# # Now normalized_features is ready to be used in machine learning models
# print(normalized_features)

/var/folders/0c/21w8m2ps1nj3wp8gv22kpyp40000gn/T/ipykernel_49319/870808706.py:88: DeprecationWarning: scipy.signal.cwt is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  wavelet_transform = scipy.signal.cwt(signal, scipy.signal.ricker, widths=np.arange(1, 31))


In [18]:
features_scaled.shape

(404, 10136)

In [24]:
len(labels)

404

In [28]:
labels

['no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'no-gesture',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'nail-fling-right',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-swipe-left',
 'finger-fling-left',
 'fin

# Random Forest

In [58]:
# save features_scaled and labels_array to "DNN data" folder
np.save("DNN data/features_scaled.npy", features_scaled)
np.save("DNN data/labels_array.npy", labels_array)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.25, random_state=42)

# Train a model
model = RandomForestClassifier(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate the model
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

# # Output the predictions and their corresponding true labels
# for i, (pred, label) in enumerate(zip(predictions, y_test)):
#     print(f"Sample {i}: Prediction = {pred}, True Label = {label}")

print(f'Support RandomForestClassifier\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Support RandomForestClassifier\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%\n')


                     precision    recall  f1-score   support

              alarm       1.00      1.00      1.00         6
finger-double-click       0.86      0.75      0.80         8
  finger-fling-down       0.00      0.00      0.00         3
  finger-fling-left       0.75      0.75      0.75         4
 finger-fling-right       0.33      0.50      0.40         4
    finger-fling-up       0.00      0.00      0.00         1
        finger-snap       1.00      1.00      1.00         2
  finger-swipe-down       0.25      0.25      0.25         4
  finger-swipe-left       1.00      0.17      0.29         6
 finger-swipe-right       0.83      0.62      0.71         8
    finger-swipe-up       0.40      0.80      0.53         5
         fist_pound       1.00      0.75      0.86         4
        mouse-click       0.50      1.00      0.67         2
         mouse-move       1.00      1.00      1.00         5
    nail-fling-down       0.60      0.60      0.60         5
    nail-fling-left    

/Users/pengyuecheng/Documents/UW/TECHIN 515/final/python/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/pengyuecheng/Documents/UW/TECHIN 515/final/python/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/pengyuecheng/Documents/UW/TECHIN 515/final/python/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

model = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))
model.fit(X_train, y_train)

# Predict and evaluate the model
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

print(f'Support Vector Machine Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Support Vector Machine Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%\n')


                 precision    recall  f1-score   support

finger-multiple       0.86      0.86      0.86         7
  finger-single       0.50      0.29      0.36         7
           fist       0.20      0.50      0.29         2
        knuckle       0.67      0.86      0.75         7
     no-gesture       0.75      1.00      0.86         6
          swipe       0.00      0.00      0.00         5

       accuracy                           0.62        34
      macro avg       0.50      0.58      0.52        34
   weighted avg       0.56      0.62      0.57        34

Support Vector Machine Model's accuracy on training set is 54.55%
Support Vector Machine Model's accuracy on test set is 61.76%



/Users/nancyluo/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(
    activation = 'relu',
    solver = 'adam',
)

model.fit(X_train, y_train)

# Predict and evaluate the model
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))


print(f'Neural Network Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Neural Network Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%\n')



                 precision    recall  f1-score   support

finger-multiple       1.00      0.86      0.92         7
  finger-single       0.75      0.43      0.55         7
           fist       0.25      1.00      0.40         2
        knuckle       0.83      0.71      0.77         7
     no-gesture       0.75      0.50      0.60         6
          swipe       0.50      0.60      0.55         5

       accuracy                           0.65        34
      macro avg       0.68      0.68      0.63        34
   weighted avg       0.75      0.65      0.67        34

Neural Network Model's accuracy on training set is 66.23%
Neural Network Model's accuracy on test set is 64.71%



/Users/nancyluo/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
!pip install micromlgen

## Create a class mapping between labels and classes

In [ ]:
from micromlgen import port


LABELS = ['alarm','finger-double-click','finger-fling-down',
          'finger-fling-left','finger-fling-right','finger-fling-up',
          'finger-snap','finger-swipe-down','finger-swipe-left','finger-swipe-right',
          'finger-swipe-up','fist_pound','mouse-click','mouse-move','nail-fling-down',
          'nail-fling-left','nail-fling-right','nail-fling-up','no-gesture','slap',
          'tap-singlefinger','typing'] #our existing labels
classMap = {} #create an empty dict
for i, label in zip(range(len(LABELS)),LABELS): #interate over the range and the labels at the same time 
  classMap[i]=label #fill our dict

print(classMap)

## Convert the model
use the random forest classifier

In [ ]:
c_code = port(RF_model,classmap=classMap) #convert our model

#Let's write it into a .h file
modelFile = open("model.h", "w")
modelFile.write(c_code)
modelFile.close()

#Let's print the size of the .h file
import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")
     